## Heater efficiency

You can simulate the index change effect from a heater in MPB

Lets assume the temperature increases by 10C (the actual increase does not matter)

**Question** 

What is the optimal waveguide width for maximum index change?

In [ ]:
from tqdm import tqdm
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import gdsfactory.simulation.modes as gm

In [ ]:
dn_dt_si = 1.87e-4
dn_dt_sio2 = 8.5e-6

In [ ]:
wg_widths = np.arange(0.4, 1.3, 0.2)
wg_widths

In [ ]:
filepath = pathlib.Path('neff_vs_temperature.csv')

In [ ]:
if filepath.exists:
    df = pd.read_csv(filepath)
    dt = 10
    
else:
    dneffs = []
    for wg_width in tqdm(wg_widths):
        dt = 0
        mode_solver_t0 = gm.get_mode_solver_rib(wg_width=wg_width, ncore=3.47 + dn_dt_si*dt, nclad=1.44 + dn_dt_sio2*dt, wg_thickness=.22, resolution=20, sy=6, sz=6, nmodes=4)
        modes = gm.find_modes(mode_solver=mode_solver_t0)
        m1 = modes[1]
        neff_t0 = m1.neff

        dt = 10
        mode_solver_t1 = gm.get_mode_solver_rib(wg_width=wg_width, ncore=3.47 + dn_dt_si*dt, nclad=1.44 + dn_dt_sio2*dt, wg_thickness=.22, resolution=20, sy=6, sz=6, nmodes=4)
        modes = gm.find_modes(mode_solver=mode_solver_t1)
        m1 = modes[1]
        neff_t1 = m1.neff

        dneff = neff_t1 - neff_t0
        dneffs.append(dneff)
        
    df = pd.DataFrame(dict(wg_width=wg_widths, dneff=dneffs))
    df.to_csv(filepath)

In [ ]:
wg_widths = df.wg_width
dneffs = df.dneff

In [ ]:
plt.plot(wg_widths, np.array(dneffs)/dt, '.-')
plt.xlabel('waveguide width (um)')
plt.ylabel('dneff')

In [ ]:
dndt=  np.array(dneffs)/dt
plt.plot(wg_widths, dndt/max(dndt) *100, '.-')
plt.title('waveguide dn/dT')
plt.xlabel('waveguide width (um)')
plt.ylabel('dn/dT (%)')